In [1]:
import tweepy
import csv
import time
import html.parser as hp
import pandas as pd
import nltk
import pickle
import numpy as np
import gensim
import re  # regular expressions (for playing with the text)
from gensim.parsing.preprocessing import STOPWORDS # common english "stop words" -- a, the, etc.
from gensim.parsing import PorterStemmer
from gensim import corpora, models
import nltk.data
import nltk

from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline
import spacy

from nltk.tokenize import RegexpTokenizer
from gensim import corpora, models
import gensim
import os
from os import path
from time import sleep
import matplotlib.pyplot as plt
import random
import wordcloud
from wordcloud import WordCloud , STOPWORDS

import re
import pickle
import gensim
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import string


In [2]:
final=pd.read_excel('Risk2010 -2018.xlsx')

In [3]:
final.shape

(131270, 5)

In [4]:
final.head()

,Unnamed: 0,filing_date,fund_CIK,principal_risks,Unnamed: 4
0,0001318148-10-001804,20101124,879569,of investing in the fund?all mutual funds tak...,NaN
1,0000356682-10-000007,20100201,356682,you could lose money on your investment in t...,NaN
2,0001193125-10-282324,20101216,1174610,of leveraged and inverse leveraged funds and ...,NaN
3,0001398344-10-000552,20100430,890453,you may lose money by investing in t...,NaN
4,0000950123-10-065147,20100713,1424958,an investment in the fund entails ri...,NaN


In [5]:
final.rename(columns={"Unnamed: 4": "col2"}, inplace=True)

In [6]:
final.head()

,Unnamed: 0,filing_date,fund_CIK,principal_risks,col2
0,0001318148-10-001804,20101124,879569,of investing in the fund?all mutual funds tak...,NaN
1,0000356682-10-000007,20100201,356682,you could lose money on your investment in t...,NaN
2,0001193125-10-282324,20101216,1174610,of leveraged and inverse leveraged funds and ...,NaN
3,0001398344-10-000552,20100430,890453,you may lose money by investing in t...,NaN
4,0000950123-10-065147,20100713,1424958,an investment in the fund entails ri...,NaN


In [7]:
final.col2.isna().sum()

115198

In [8]:
final.replace(np.nan, '', regex=True,inplace=True)

In [9]:
final.head()

,Unnamed: 0,filing_date,fund_CIK,principal_risks,col2
0,0001318148-10-001804,20101124,879569,of investing in the fund?all mutual funds tak...,
1,0000356682-10-000007,20100201,356682,you could lose money on your investment in t...,
2,0001193125-10-282324,20101216,1174610,of leveraged and inverse leveraged funds and ...,
3,0001398344-10-000552,20100430,890453,you may lose money by investing in t...,
4,0000950123-10-065147,20100713,1424958,an investment in the fund entails ri...,


In [10]:
#final['Text']=final.principal_risks+" "+final.col2
final['Text'] = final['principal_risks'].str.cat(final['col2'],sep=" ")

In [11]:
final.head()

,Unnamed: 0,filing_date,fund_CIK,principal_risks,col2,Text
0,0001318148-10-001804,20101124,879569,of investing in the fund?all mutual funds tak...,,of investing in the fund?all mutual funds tak...
1,0000356682-10-000007,20100201,356682,you could lose money on your investment in t...,,you could lose money on your investment in t...
2,0001193125-10-282324,20101216,1174610,of leveraged and inverse leveraged funds and ...,,of leveraged and inverse leveraged funds and ...
3,0001398344-10-000552,20100430,890453,you may lose money by investing in t...,,you may lose money by investing in t...
4,0000950123-10-065147,20100713,1424958,an investment in the fund entails ri...,,an investment in the fund entails ri...


In [12]:
mylist=['lose','losing','lost','loses','loss']
value_list=[]
for row in final['Text']:
    before_one_word=[]
    before_two_word=[]
    before_words=[]
    row=row.translate(str.maketrans('', '', string.punctuation))
    words = row.lower().split()
    t = [i for i, x in enumerate(words) if x == "money"]
    for d in t:
        before_one_word.append(words[d-1])
        before_two_word.append(words[d-2])
    before_words=before_one_word+before_two_word
    if len([i for i in mylist if i in before_words])>0:
        value_list.append(1)
    else:
        value_list.append(0)
        #if ('lose' == (words[d-1]) or 'lose' == words[d-2]) or ('losing' == (words[d-1]) or 'losing' == (words[d-2])) or ('lost' == (words[d-1]) or 'lost' == (words[d-2])) or ('loses' == (words[d-1]) or 'loses' == (words[d-2])):
#             val=1
            #value_list.append(1)
#         else:
#             val=0
# #             #value_list.append(0)
#     value_list.append(val)

In [13]:
final['comp_money']=''

In [14]:
final['comp_money']=value_list

In [15]:
final.head()

,Unnamed: 0,filing_date,fund_CIK,principal_risks,col2,Text,comp_money
0,0001318148-10-001804,20101124,879569,of investing in the fund?all mutual funds tak...,,of investing in the fund?all mutual funds tak...,1
1,0000356682-10-000007,20100201,356682,you could lose money on your investment in t...,,you could lose money on your investment in t...,1
2,0001193125-10-282324,20101216,1174610,of leveraged and inverse leveraged funds and ...,,of leveraged and inverse leveraged funds and ...,1
3,0001398344-10-000552,20100430,890453,you may lose money by investing in t...,,you may lose money by investing in t...,1
4,0000950123-10-065147,20100713,1424958,an investment in the fund entails ri...,,an investment in the fund entails ri...,1


In [37]:
final.comp_money.value_counts()

1    121467
0      9803
Name: comp_money, dtype: int64

In [38]:
mylist=['lose','losing','lost','loses','loss']
value_list=[]
for row in final['Text']:
    before_one_word=[]
    before_two_word=[]
    before_three_word=[]
    before_four_word=[]
    before_five_word=[]
    before_words=[]
    row=row.translate(str.maketrans('', '', string.punctuation))
    words = row.lower().split()
    t = [i for i, x in enumerate(words) if (x == "investment" or x=="investing" or x == "investments")]
    for d in t:
        before_one_word.append(words[d-1])
        before_two_word.append(words[d-2])
        if d >=1:
            before_three_word.append(words[d-3])
            before_four_word.append(words[d-4])
            before_five_word.append(words[d-5])
    before_words=before_one_word+before_two_word+before_three_word+before_four_word+before_five_word
    if len([i for i in mylist if i in before_words])>0:
        value_list.append(1)
    else:
        value_list.append(0)
        #if ('lose' == (words[d-1]) or 'lose' == words[d-2]) or ('losing' == (words[d-1]) or 'losing' == (words[d-2])) or ('lost' == (words[d-1]) or 'lost' == (words[d-2])) or ('loses' == (words[d-1]) or 'loses' == (words[d-2])):
#             val=1
            #value_list.append(1)
#         else:
#             val=0
# #             #value_list.append(0)
#     value_list.append(val)

In [39]:
final['comp_invest']=''

In [40]:
final['comp_invest']=value_list

In [41]:
final.head()

,Unnamed: 0,filing_date,fund_CIK,principal_risks,col2,Text,comp_money,comp_invest
0,0001318148-10-001804,20101124,879569,of investing in the fund?all mutual funds tak...,,of investing in the fund?all mutual funds tak...,1,1
1,0000356682-10-000007,20100201,356682,you could lose money on your investment in t...,,you could lose money on your investment in t...,1,1
2,0001193125-10-282324,20101216,1174610,of leveraged and inverse leveraged funds and ...,,of leveraged and inverse leveraged funds and ...,1,0
3,0001398344-10-000552,20100430,890453,you may lose money by investing in t...,,you may lose money by investing in t...,1,1
4,0000950123-10-065147,20100713,1424958,an investment in the fund entails ri...,,an investment in the fund entails ri...,1,0


In [42]:
final.comp_invest.value_counts()

1    90275
0    40995
Name: comp_invest, dtype: int64

In [43]:
der=['derivative','derivatives']

count_der=0
cnt_der=[]
for row in final['Text']:
    row=row.translate(str.maketrans('', '', string.punctuation))
    words = row.lower().split()
    #count_neg.append(len([i for i in tokens1 if i in list_Negative]))
    count_der = len([i for i in words if i in der])
    cnt_der.append(count_der)

In [44]:
final['Cnt_Derivative']=''
final['Cnt_Derivative']=cnt_der

In [45]:
final.head()

,Unnamed: 0,filing_date,fund_CIK,principal_risks,col2,Text,comp_money,comp_invest,Cnt_Derivative
0,0001318148-10-001804,20101124,879569,of investing in the fund?all mutual funds tak...,,of investing in the fund?all mutual funds tak...,1,1,7
1,0000356682-10-000007,20100201,356682,you could lose money on your investment in t...,,you could lose money on your investment in t...,1,1,0
2,0001193125-10-282324,20101216,1174610,of leveraged and inverse leveraged funds and ...,,of leveraged and inverse leveraged funds and ...,1,0,4
3,0001398344-10-000552,20100430,890453,you may lose money by investing in t...,,you may lose money by investing in t...,1,1,0
4,0000950123-10-065147,20100713,1424958,an investment in the fund entails ri...,,an investment in the fund entails ri...,1,0,10


In [46]:
final.Cnt_Derivative.describe()

count    131270.000000
mean          3.921223
std           5.576831
min           0.000000
25%           0.000000
50%           0.000000
75%           7.000000
max          59.000000
Name: Cnt_Derivative, dtype: float64

In [47]:
#final.to_csv('Risk_Compliance_Derivative2010-2018.csv', index=False)

In [48]:
#final.to_excel('Risk_Compliance_Derivative2010-2018.xlsx', index=False)

In [49]:
final.Cnt_Derivative.value_counts().head()

0    67733
5     6178
6     5566
7     5442
8     5142
Name: Cnt_Derivative, dtype: int64

In [50]:
final.shape

(131270, 9)

In [51]:
final.comp_money.value_counts()

1    121467
0      9803
Name: comp_money, dtype: int64

In [52]:
final.comp_invest.value_counts()

1    90275
0    40995
Name: comp_invest, dtype: int64

In [53]:
final.head()

,Unnamed: 0,filing_date,fund_CIK,principal_risks,col2,Text,comp_money,comp_invest,Cnt_Derivative
0,0001318148-10-001804,20101124,879569,of investing in the fund?all mutual funds tak...,,of investing in the fund?all mutual funds tak...,1,1,7
1,0000356682-10-000007,20100201,356682,you could lose money on your investment in t...,,you could lose money on your investment in t...,1,1,0
2,0001193125-10-282324,20101216,1174610,of leveraged and inverse leveraged funds and ...,,of leveraged and inverse leveraged funds and ...,1,0,4
3,0001398344-10-000552,20100430,890453,you may lose money by investing in t...,,you may lose money by investing in t...,1,1,0
4,0000950123-10-065147,20100713,1424958,an investment in the fund entails ri...,,an investment in the fund entails ri...,1,0,10


In [54]:
final.Cnt_Derivative.describe()

count    131270.000000
mean          3.921223
std           5.576831
min           0.000000
25%           0.000000
50%           0.000000
75%           7.000000
max          59.000000
Name: Cnt_Derivative, dtype: float64

In [57]:
final['Compliance']=final['comp_money'] + final['comp_invest']

In [58]:
final.Compliance.value_counts()

2    88416
1    34910
0     7944
Name: Compliance, dtype: int64

In [59]:
final.shape

(131270, 10)

In [60]:
88416+34910

123326

In [61]:
123326/131270

0.9394835072750819